In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import annotations

import jax.numpy as jnp
import jax
from pathlib import Path
from diffwake.diffwake_jax.model_agnostic import load_input, create_state, turbine_powers, alter_yaw_angles
from diffwake.diffwake_jax.sim_agnostic import simulate, simulate_simp
from diffwake.diffwake_jax.turbine.operation_models import power as power_fn
from diffwake.diffwake_jax.util_agnostic import set_precision
from floris import FlorisModel, TimeSeries
import numpy as np
import time


# helper functions
def absolute_percentage_difference(a, b):
    return jnp.abs((a - b) / b) * 100


DTYPE = set_precision("float32")
print(f"Current floating point precision: {DTYPE}")

Current floating point precision: <class 'jax.numpy.float32'>


## DiffWake yawed vs unyawed comparisons

In [103]:
# 1. Define paths to configuration files
wake_model = "gauss"
data_dir = Path("data/horn")
farm_cfg_path = data_dir / f"{wake_model}_hornsRev.yaml"
turbine_cfg_path = data_dir / "vestas_v802MW.yaml"

# 2. Load the base configuration
cfg = load_input(farm_cfg_path, turbine_cfg_path)

# 3. Define a simple 3-turbine layout (aligned along the X-axis)
ws = 13.5
wd = 270.0
ti_val = 0.06
layout_x = jnp.array([0.0, 400.0, 800.0], dtype=DTYPE)
layout_y = jnp.array([0.0, 0.0, 0.0], dtype=DTYPE)

# 4. Define flow conditions
wind_speed = jnp.array([ws], dtype=DTYPE)
wind_dir = jnp.array([jnp.deg2rad(wd)], dtype=DTYPE) # Wind from West
ti = jnp.array([ti_val], dtype=DTYPE)

# 5. Update the configuration
cfg = cfg.set(
    layout_x=layout_x,
    layout_y=layout_y,
    wind_speeds=wind_speed,
    wind_directions=wind_dir,
    turbulence_intensities=ti
)

# 6. Create the simulation state
state = create_state(cfg)

# --- Baseline: Unyawed Farm ---
yaw_angles_unyawed = jnp.zeros((1, 3), dtype=DTYPE) # (n_findex, n_turbines)
state_unyawed = alter_yaw_angles(yaw_angles_unyawed, state)
result_unyawed = simulate(state_unyawed)
unyawed_turbine_powers = power_fn(power_thrust_table=state.farm.power_thrust_table,
                                  velocities=result_unyawed.u_sorted,
                                  air_density=state.flow.air_density,
                                  yaw_angles=state.farm.yaw_angles,)
unyawed_farm_power = jnp.sum(unyawed_turbine_powers)/1e6 # convert to megawatts


# state_final_unyawed = state_unyawed.replace(flow=state_unyawed.flow.replace(u=result_unyawed.u_sorted))
# powers_unyawed = turbine_powers(state_final_unyawed)[0] # First (and only) wind condition

# # --- Case: Yawed Farm ---
y1, y2, y3 = 35.0, 10.0, 0.0
yaw_angles_deg = jnp.array([[y1, y2, y3]], dtype=DTYPE) # Try steering wakes away from downstream turbines
yaw_angles_rad = jnp.deg2rad(yaw_angles_deg)

state_yawed = alter_yaw_angles(yaw_angles_rad, state)
result_yawed = simulate(state_yawed)
yawed_turbine_powers = power_fn(power_thrust_table=state_yawed.farm.power_thrust_table,
                                  velocities=result_yawed.u_sorted,
                                  air_density=state_yawed.flow.air_density,
                                  yaw_angles=state_yawed.farm.yaw_angles,)
yawed_farm_power = jnp.sum(yawed_turbine_powers)/1e6


# FLORIS YAW
fmodel = FlorisModel(rf"../floris/examples/inputs/{wake_model}_vesta.yaml")

floris_yaw_angles = np.array([y1, y2, y3]).reshape(1, -1)
time_series = TimeSeries(
    wind_directions=np.array([wd]),
    wind_speeds=np.array([ws]),
    turbulence_intensities=ti_val,
)

fmodel.set(wind_data=time_series,
           layout_x=np.array([0.0, 400.0, 800.0]),
           layout_y=np.array([0.0, 0.0, 0.0]),
           yaw_angles=np.zeros_like(floris_yaw_angles))
fmodel.run()

# baseline case
floris_baseline_power = fmodel.get_farm_power() / 1e6
turbine_powers_floris = fmodel.get_turbine_powers()

# yawed case
fmodel.set(yaw_angles=floris_yaw_angles)
fmodel.run()
floris_yawed_power = fmodel.get_farm_power() / 1e6
turbine_powers_floris_yawed = fmodel.get_turbine_powers()

In [104]:
print(f"----- FLORIS results -----")
print(f"Baseline farm power: {floris_baseline_power[0]:.6f} MW")
print(f"Turbine powers: {[i.item() for i in turbine_powers_floris[0]]}" )
print(f"Yawed farm power: {floris_yawed_power[0]:.6f} MW")
print(f"Turbine powers: {[i.item() for i in turbine_powers_floris_yawed[0]]}\n" )


print(f"----- DiffWake results -----")
print(f"Baseline farm power: {unyawed_farm_power:.6f} MW")
print(f"Turbine powers: {[i.item() for i in unyawed_turbine_powers[0]]}" )
print(f"Yawed farm power: {yawed_farm_power:.6f} MW")
print(f"Turbine powers: {[i.item() for i in yawed_turbine_powers[0]]}" )


print("\n---------- Differences ----------")
print(f"Absolute baseline power difference (GCH) = {absolute_percentage_difference(floris_baseline_power, unyawed_farm_power)[0]:.8f}%")
print(f"Absolute yawed power difference (GCH) = {absolute_percentage_difference(floris_yawed_power, yawed_farm_power)[0]:.8f}%")

----- FLORIS results -----
Baseline farm power: 4.836709 MW
Turbine powers: [1972125.6058977128, 1663348.6365516074, 1201235.1647396581]
Yawed farm power: 5.141033 MW
Turbine powers: [1843010.5386453331, 1838930.8363115978, 1459091.9864042029]

----- DiffWake results -----
Baseline farm power: 4.836627 MW
Turbine powers: [1972125.6058977128, 1663348.6634779854, 1201152.8269721586]
Yawed farm power: 5.231695 MW
Turbine powers: [1843010.5386453327, 1793881.6633172126, 1594803.2394806168]

---------- Differences ----------
Absolute baseline power difference (GCH) = 0.00170182%
Absolute yawed power difference (GCH) = 1.73293880%
